In [1]:
import tensorflow as tf
import numpy as np
import unicodedata
import codecs

#set hyperparameters
max_len = 60
step = 2
num_units = 128
learning_rate = 0.001
batch_size = 200
epoch = 60
temperature = 0.5

In [2]:
def read_data(file_name):
    '''
     open and read text file
    '''
    text = codecs.open(file_name, encoding='utf-8', mode='r').read()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')
    text = text.replace("*", "")
    text = text.replace("_", "")
    text = text.replace("-", "")
    return text.lower()

In [3]:
def featurize(text):
    '''
     featurize the text to train and target dataset
    '''
    unique_chars = list(set(text))
    len_unique_chars = len(unique_chars)

    input_chars = []
    output_char = []

    for i in range(0, len(text) - max_len, step):
        input_chars.append(text[i:i+max_len])
        output_char.append(text[i+max_len])

    train_data = np.zeros((len(input_chars), max_len, len_unique_chars))
    target_data = np.zeros((len(input_chars), len_unique_chars))

    for i , each in enumerate(input_chars):
        for j, char in enumerate(each):
            train_data[i, j, unique_chars.index(char)] = 1
        target_data[i, unique_chars.index(output_char[i])] = 1
    return train_data, target_data, unique_chars, len_unique_chars


In [4]:
def rnn(x, weight, bias, len_unique_chars):
    '''
     define rnn cell and prediction
    '''
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, len_unique_chars])
    x = tf.split(x, max_len, 0)

    cell = tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction

In [5]:
def sample(predicted):
    '''
     helper function to sample an index from a probability array
    '''
    exp_predicted = np.exp(predicted/temperature)
    predicted = exp_predicted / np.sum(exp_predicted)
    probabilities = np.random.multinomial(1, predicted, 1)
    return probabilities


In [6]:
def run(train_data, target_data, unique_chars, len_unique_chars):
    '''
     main run function
    '''
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)

    num_batches = int(len(train_data)/batch_size)

    for i in range(epoch):
        print "----------- Epoch {0}/{1} -----------".format(i+1, epoch)
        count = 0
        for _ in range(num_batches):
            train_batch, target_batch = train_data[count:count+batch_size], target_data[count:count+batch_size]
            count += batch_size
            sess.run([optimizer] ,feed_dict={x:train_batch, y:target_batch})
        #get on of training set as seed
        seed = train_batch[:1:]
        if (i + 1) % 20 == 0:
#         save_path = saver.save(sess, "./tmp/rnn_model" + str(i) + ".ckpt")
            save_path = saver.save(sess, "./tmp/p_and_p/rnn_model" + str(i) + ".ckpt")

        #to print the seed 40 characters
        seed_chars = ''
        for each in seed[0]:
                seed_chars += unique_chars[np.where(each == max(each))[0][0]]
        print "Seed:", seed_chars

        #predict next 1000 characters
        for i in range(1000):
            if i > 0:
                remove_fist_char = seed[:,1:,:]
                seed = np.append(remove_fist_char, np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
            predicted = sess.run([prediction], feed_dict = {x:seed})
            predicted = np.asarray(predicted[0]).astype('float64')[0]
            probabilities = sample(predicted)
            predicted_chars = unique_chars[np.argmax(probabilities)]
            seed_chars += predicted_chars
        print 'Result:', seed_chars
    sess.close()


In [7]:
def convert_to_seed(text, unique_chars):
    len_uniques = len(unique_chars)
    letters = []
    for i in text:
        letter = np.zeros((1, len_uniques))
        for j in range(len(unique_chars)):
            if i == unique_chars[j]:
                letter[0, j] = 1.
        letters.append(letter)
    return np.array([np.vstack(letters)])
    

In [8]:
def restore_model():
    text = read_data('p_and_p.txt')
#     text = read_data('shakespeare.txt')
    train_data, target_data, unique_chars, len_unique_chars = featurize(text)
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)
    saver.restore(sess, "./tmp/rnn_model1.ckpt")
    
    return unique_chars, prediction, sess, x, len_unique_chars

In [9]:
def predict(text, unique_chars, prediction, sess, x, len_unique_chars):
    
    text_head = ''
    pad_head = 0
    if len(text) > 40:
        text_head = text[:-40]        
        text = text[-40:]
    elif len(text) < 40:
        pad_head  = 40 - len(text)
        text = ' ' * pad_head + text
        
    seed = convert_to_seed(text, unique_chars)
    #to print the seed 40 characters
    seed_chars = ''
    for each in seed[0]:
            seed_chars += unique_chars[np.where(each == max(each))[0][0]]
    print "Seed:", seed_chars

    
    #predict next 1000 characters
    for i in range(50):
        if i > 0:
            remove_fist_char = seed[:,1:,:]
            seed = np.append(remove_fist_char, 
                             np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
        predicted = sess.run([prediction], feed_dict = {x:seed})
        predicted = np.asarray(predicted[0]).astype('float64')[0]
        probabilities = sample(predicted)
        predicted_chars = unique_chars[np.argmax(probabilities)]
        seed_chars += predicted_chars
        if predicted_chars == '.':
            return (text_head + seed_chars)[pad_head:]

    

In [10]:
# unique_chars, predictions, sess, x, len_unique_chars = restore_model()
# predict('out fane with you ', unique_chars, predictions, sess, x, len_unique_chars)

In [11]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# run(train_data, target_data, unique_chars, len_unique_chars)

In [12]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# train_batch, target_batch = train_data[1:5], target_data[1:5]
# seed = train_batch[:1:]
# print seed

In [13]:
# np.zeros((1, 5))

In [14]:
# def convert_to_seed(text, unique_chars):
#     len_uniques = len(unique_chars)
#     letters = []
#     for i in text:
#         letter = np.zeros((1, len_uniques))
#         for j in range(len(unique_chars)):
#             if i == unique_chars[j]:
#                 letter[0, j] = 1.
#         letters.append(letter)
#     return np.array([np.vstack(letters)])
    

In [15]:
# convert_to_seed('hello', unique_chars).shape

In [16]:
text = read_data('p_and_p.txt')
train_data, target_data, unique_chars, len_unique_chars = featurize(text)
run(train_data, target_data, unique_chars, len_unique_chars)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

----------- Epoch 1/60 -----------
Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at preressed dere the har asteromed the reatiathed of his itter elizabether forance surt of her and the how losed her dare the fore to mist as all and the mexter and was eas alled the have to dered ave frem to elizabethor her garess with to hem as in the hir in her ally ave bether andse of the fored be him of the mor. and as wish to mere whot every and the rester sed tiok of her ther fere the gore the was the bever of comerter sher mart of hir maratly coringed were the for to ther to have ther mr. elizabeth, bether were of his was lither most be the beented the gare, but lens with her aver to her mert of hir wish the forme from tore the 

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at possible of merything to recovery to her family. indervany of him had been in
nother of assure of the arrival to himself, that she was to the friend of
them to all the course of his dissister was exceeding at her
susting that i was good a manners. he was in her extents and their restifior of his sister, and he have expected to her more of her comfaranted, and all the consent of very arrival and likely to least estain. the ladyship was propession fur her own to the world anything and and as they was now to her own so entorratured on his offerspect to the world in her inthing to miss
darcy say not and circumstances of her what ever sand woman the looked and almost and come to pose them to gratinge of her sisters of in her behavions and them with her coming as them were, to desire?

you are you and your answer. i have gave her coming as to be sister, and he

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at persons, and the ladies after the entrest the too married, and her affectionate was as all in a manner of her own and manners as it so friends, her satisfaction which he was off and the latter of him
and her doud.

her announted him from the excession of them, in a week instance. the other contents of the morning and in her away.

you will be overto a ressible of little or and him after the former of nothing that he have been to meet to elizabeth's were not of his appleasing them and all the formert of her doubt. the course of the letter was soon and instantly sporr abode of stanishment to more the present first the marriage
of the rest of the confession of his marriage. the carriage of them even to see him own the winter of them evening, it was to entrarnce. your father was so strong to yourself on the evening, and you should not be so frequent as her b

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at present as to ask the only explained for her own graduress of his family, and all the compliments of the
satisfacted
as a such a mind of the door, they were all the companion of
mr. darcy will not be done and satisfied on the young woman where they was not of him, which they were all to thing with him to have all the mort for of him; and she she even to be done and all the party of the reasonable as their course of the account of her beand to her own which he was ristently sometimes and her determined to be designed so for her former. the affectionation which her resolution of the marriage was to be disturbed. and they may be expectedly, and i will be a most a come to her morning at the
satisfaction of her morning of the particulars always account of the
persons were at longbourn of spend of resolution to conversation, he had a manners of her own myself 

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at persons was all think of
her sisters and all her sisters and her sister declared to her every of the others. i dare say. i am expected this were so all her distanced at the own remarkable to her as has been intentionation of her command. he was to go would have been at lady, and he had not been up to her other of their consert and her mother of the house, i am sure so fear on what she was engaged to meet to her sister which a letter was all the same time with her of the acchuding mr. darcy could not have been little a place which had been her love him as much marriage in her morning, whenever mr. darcy had all the sisters and was extenting againatione to express the marriage of her daughter, when my dear sir, and her resentment was able to confess them, they were all in her surprised on by such a character which in his achings. i had not believe her fath

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at persons nor having the confest cheelfrance of her ladyship, and her coming on the sone too much to any the prospiction of some proudly recollecter consequence of her sister was all the parsonage would have been all the way. elizabeth had been gentled away, and with her the servant was off correspingness of the room. i was all laughed the room, and of the letter stead.

i have allowed them to him so situation, when they were all the sorr. to mr. darcy should
be so all so from the morning but him formed and desting to make
her matters and her evering them, and the morning. they were so sortly to any officers. and all the disliking to as she was to longbourn of of their own repeating. they were all and confession of their companion of in her own rembingeated any children was surprised, and her seeing with her away.

but i hope it would be an accomption firs

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at present in her daughter, and the charming a man who had a great doubt. he was prevented is the
feelings of her to believe me. i can be to be so far for her respectabless and belief of all she was put seeed a sincerelver was still for her followed to every thing to her and at
an hours in confession to her sisters would be a word, and as she was so all of the hould a preadedson, and her daughters.

i love without any particulars of him to expressed the wilder. i cannot all the particulars at last? i am all afforted with the room, when you look as i could. they were all restinence. his disgracceror which had done so all of her daughters. we lever in a moment which had soes to be delighed to me
mery in a hamp her sisters was supported to elizabeth. the commended with him as passing by the same
time, she was reasiver, for the hopeless were to be an
earness is

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at persons were likely to near the servants she would have been an acknowledgements and have been even to see miss bennet. but when i must have been in town meeting and more return. the cheeck delighted of his own
was as he was silent could not heard to her answer. she was well wish for her reflecte. i knew that she was to be an actions or her family had been tranthing. i did not see his lived in a daughters. elizabeth was soon and at
else to elizabeth, and the greatest probably she was greatent against him as to mrs. collins accular me that their neighbours which to be the invitiable in her morning, and the former. what it it was expected by a singlition of her confess. this was a sincerel of the friends to be designed her.

you are about intolest before in her room, and while i now do, i am sure i deligited by this man when she should not be so at the par

Seed: conducted herself; and she condescended to wait
on them at p
Result: conducted herself; and she condescended to wait
on them at persons was now downing to the liberte to her mother. when child in returning that mrs. bennet, i am affectly a family. at least after mr. bingley's morning, and there was about his own,
to speak on her applicent to her room. and were in a things, and he had been in town,
when i should be so friend; and as much as it of the rest of the same time does between themselves of her confessance, and there was him will be to be supposed to make him and all the servant was
convinced it she mettionefully herself.

i do not be soon happiness of his designed would be to be done.

with theight of her into whils
featers, he was not sortiemed one to a sincerels expressed them at the althir at the saturnce of her petfectly could not have been loss to situren the conversation was able to say, or any of her drawn the sisters of the acquaintance as her intradugity was wish